---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [26]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [39]:
def answer_one(debug=False):
        
    # Your Code Here
    df_G = pd.read_csv('Employee_Movie_Choices.txt', sep='\t')
    
    G = nx.from_pandas_dataframe(df_G, '#Employee', 'Movie')
    
    return G  # Your Answer Here

# answer_one(True)

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [36]:
def answer_two(debug=False):
    
    # Your Code Here
    G = answer_one()
    
    for node in G.nodes():
        if node in employees:
            G.add_node(node, type='employee')
        elif node in movies:
            G.add_node(node, type='movie')
        else:
            print("Unknow node type: {}".format(node))
            return None
    
    return G     # Your Answer Here

# answer_two(True).nodes(data=True)

[('Andy', {'type': 'employee'}),
 ('Anaconda', {'type': 'movie'}),
 ('Mean Girls', {'type': 'movie'}),
 ('The Matrix', {'type': 'movie'}),
 ('Claude', {'type': 'employee'}),
 ('Monty Python and the Holy Grail', {'type': 'movie'}),
 ('Snakes on a Plane', {'type': 'movie'}),
 ('Frida', {'type': 'employee'}),
 ('The Shawshank Redemption', {'type': 'movie'}),
 ('The Social Network', {'type': 'movie'}),
 ('Georgia', {'type': 'employee'}),
 ('Joan', {'type': 'employee'}),
 ('Forrest Gump', {'type': 'movie'}),
 ('Kung Fu Panda', {'type': 'movie'}),
 ('Lee', {'type': 'employee'}),
 ('Pablo', {'type': 'employee'}),
 ('The Dark Knight', {'type': 'movie'}),
 ('Vincent', {'type': 'employee'}),
 ('The Godfather', {'type': 'movie'})]

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [35]:
def answer_three(debug=False):
        
    # Your Code Here
    P = bipartite.weighted_projected_graph(answer_two(), employees)
    
    return P   # Your Answer Here

# answer_three(True).edges(data=True)

[('Vincent', 'Frida', {'weight': 2}),
 ('Vincent', 'Pablo', {'weight': 1}),
 ('Claude', 'Georgia', {'weight': 3}),
 ('Claude', 'Andy', {'weight': 1}),
 ('Joan', 'Andy', {'weight': 1}),
 ('Joan', 'Lee', {'weight': 3}),
 ('Lee', 'Andy', {'weight': 1}),
 ('Pablo', 'Frida', {'weight': 2}),
 ('Pablo', 'Andy', {'weight': 1}),
 ('Frida', 'Andy', {'weight': 1}),
 ('Georgia', 'Andy', {'weight': 1})]

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [65]:
def answer_four():

    def common_movies(row):
        for e1, e2, weight in P.edges(data=True):
            if e1 > e2:
                e1, e2 = e2, e1
            if (e1 == row['Employee1']) & (e2 == row['Employee2']):
                return weight['weight']
        return 0
            
    # Your Code Here
    P = answer_three()
    
    df_ER = pd.read_csv('Employee_Relationships.txt', sep='\t', 
                        header=None, names=['Employee1', 'Employee2', 'Score'])
    
    df_ER_Movie = df_ER
    df_ER_Movie['Common'] = df_ER_Movie.apply(common_movies, axis=1)
    
    return df_ER_Movie['Score'].corr(df_ER_Movie['Common'])    # Your Answer Here

# answer_four()

0.78839622217334748